# Calculo de estados del Helio mediante el metodo de Hartree Fock

Nicolas Unger

In [1]:
import numpy as np
import sympy as sp
from scipy.linalg import eigh
import matplotlib.pyplot as plt

# Estética
#sp.init_printing()
#%matplotlib inline

In [9]:
nsize = 4

alpha = np.zeros((nsize)) # Alphas de la base gaussiana
Ci = np.zeros((nsize)) # Coeficientes para cada elemento de la base
Efund = []

Spq = np.zeros((nsize,nsize)) # Overlap
Hpq = np.zeros((nsize,nsize)) # Hamiltoniano
Fpq = np.zeros((nsize,nsize)) # Operador de Fock
F2 = np.zeros((nsize,nsize))

In [10]:
# Parametros alpha para construir la funcion de onda

# Los del Thijssen
alpha[0] = 0.297104
alpha[1] = 1.236745
alpha[2] = 5.749982
alpha[3] = 38.216677

# Los de Mitnik
alpha[0] = 0.298073
alpha[1] = 1.242567
alpha[2] = 5.782948
alpha[3] = 38.47497

# Para el hidrogeno
alpha[0] = 13.00773
alpha[1] = 1.962079
alpha[2] = 0.444529
alpha[3] = 0.1219492

In [11]:
# Funcion de onda con base gaussiana

def psi(alpha,Ci,x):
    suma = 0
    for i in range(nsize):
        suma += Ci*np.exp(-alpha[i]*x**2)
    return suma

In [12]:
# Funcion para calcular el resultado de las integrales J y K para este caso

def Qprqs(p,r,q,s,alpha):
    num = 2 * np.pi ** (5./2.)
    den1 = (alpha[p]+alpha[q])*(alpha[r]+alpha[s])
    den2 = np.sqrt(alpha[p]+alpha[q]+alpha[r]+alpha[s])
    return num/(den1*den2)

### Genero la matriz de overlap calculando los overlaps de la base Gaussiana

Saco el overlap calculando $\left <\chi_p\middle|\chi_q\right >$

\begin{equation}
\left <\chi_p\middle|\chi_q\right > = \int_0^\infty e^{-\alpha_p r^2} e^{-\alpha_q r^2} d^3\boldsymbol{r} = \int_0^\infty e^{-(\alpha_p + \alpha_q) r^2} d^3\boldsymbol{r}
\end{equation}

In [13]:
# Calculo el overlap entre la base de funciones

r = sp.Symbol('r',positive=True)
a = sp.Symbol('a',positive=True)
b = sp.Symbol('b',positive=True)
z = sp.Symbol('z',positive=True)

# La integral mas la contribucion esferica
sp.simplify(sp.integrate( r**2 * sp.exp(-(a+b)*r**2) , (r,0,sp.oo))) * 4*sp.pi

pi**(3/2)/(a + b)**(3/2)

In [14]:
# Genero la Matriz Spq
# Uso el resultado que me dio integrando el overlap de dos gaussianas.

for p in range(nsize):
    for q in range(p,nsize):
        Spq[p,q] = np.pi**(3./2.) / (alpha[p]+alpha[q])**(3./2.)
        Spq[q,p] = Spq[p,q] # Reflejo los valores ya que Spq es simetrica
Spq

array([[  4.19640644e-02,   9.61391815e-02,   1.12857904e-01,
          1.17042513e-01],
       [  9.61391815e-02,   7.16316708e-01,   1.49147774e+00,
          1.85084232e+00],
       [  1.12857904e-01,   1.49147774e+00,   6.64247101e+00,
          1.30602054e+01],
       [  1.17042513e-01,   1.85084232e+00,   1.30602054e+01,
          4.62286682e+01]])

### Genero la matriz del Hamiltoniano

Hpq esta compuesta por la parte cinetica y la parte potencial

Cinetica:

\begin{equation}
\left <\chi_p\middle| -\frac{1}{2} \nabla^2 \middle|\chi_q\right > = \int_0^\infty e^{-\alpha_p r^2} -\frac{1}{2} \nabla^2 e^{-\alpha_q r^2} d^3\boldsymbol{r}
\end{equation}

In [15]:
# Parte cinectica
# Calculo cuanto da el overlap entre los elementos de la base

sp.simplify(sp.integrate( -0.5 * r**2 * sp.exp(-a*r**2) * (sp.diff(sp.exp(-b*r**2),r,r) + (2/r)*sp.diff(sp.exp(-b*r**2),r)), (r,0,sp.oo))) * 4*sp.pi

3.0*pi**(3/2)*a*b/(a + b)**(5/2)

Potencial:

\begin{equation}
\left <\chi_p\middle| -\frac{Z}{r} \middle|\chi_q\right > = \int_0^\infty e^{-\alpha_p r^2} \left(-\frac{Z}{r}\right) e^{-\alpha_q r^2} d^3\boldsymbol{r}
\end{equation}

In [16]:
# Parte Potencial
# Calculo cuanto da el overlap entre los elementos de la base

sp.simplify(sp.integrate( sp.exp(-(a)*r**2) * -(z*r) * sp.exp(-(b)*r**2), (r,0,sp.oo)) * 4*sp.pi )

-2*pi*z/(a + b)

In [17]:
# Valor medio de la energia cinetica

Z = 1

def Tpq(alpha,p,q):
    num = 3 * np.pi**(3./2.) * alpha[p] * alpha[q]
    den = (alpha[p] + alpha[q])**(5./2.)
    return num/den

def Vpq(alpha,p,q):
    return -(2*Z*np.pi) / (alpha[p] + alpha[q])

for p in range(nsize):
    for q in range(p,nsize):
        Hpq[p,q] = Tpq(alpha,p,q) + Vpq(alpha,p,q)
        Hpq[q,p] = Hpq[p,q]
        
Hpq

array([[  0.57726847,   0.07200247,  -0.3215405 ,  -0.43612626],
       [  0.07200247,   0.50704993,  -0.98918483,  -2.37741992],
       [ -0.3215405 ,  -0.98918483,  -2.63808243,  -7.34221693],
       [ -0.43612626,  -2.37741992,  -7.34221693, -17.30516271]])

## Comienzo la iteracion

In [782]:
#Ci = [0,0,1.,1.] # Valores iniciales para los coeficientes
#Ci = [ 0.1226577 ,  0.45030895,  0.44547247,  0.28272299]

In [828]:
Ci

array([-0.06643041, -0.13252026, -0.07542911, -0.11656649])

In [829]:
# Construyo el operador de Fock
# Primero hago la suma

F2 = np.zeros((nsize,nsize))

for p in range(nsize):
    for q in range(p,nsize):
        suma = 0
        for r in range(nsize):
            for s in range(nsize):
                suma += Qprqs(p,r,q,s,alpha)*Ci[r]*Ci[s]
        F2[p,q] = suma
        F2[q,p] = F2[p,q]
        
F2

array([[  0.03202547,   0.07240888,   0.08472162,   0.08779394],
       [  0.07240888,   0.50433582,   1.00469629,   1.22734165],
       [  0.08472162,   1.00469629,   3.89287277,   6.99390228],
       [  0.08779394,   1.22734165,   6.99390228,  19.81476637]])

In [830]:
for p in range(nsize):
    for q in range(p,nsize):
        Fpq[p,q] = Hpq[p,q] + F2[p,q]
        Fpq[q,p] = Fpq[p,q]
        
Fpq

array([[ 0.60929393,  0.14441135, -0.23681887, -0.34833232],
       [ 0.14441135,  1.01138575,  0.01551146, -1.15007827],
       [-0.23681887,  0.01551146,  1.25479034, -0.34831465],
       [-0.34833232, -1.15007827, -0.34831465,  2.50960366]])

In [831]:
# Saco autovectores y autovalores

Energias, Cnuevo = eigh(Fpq,Spq,type=1)
Ci = Cnuevo[:,0]
Ci

array([-0.06492225, -0.13068511, -0.0701039 , -0.11847642])

In [832]:
Energias

array([  1.50552846e-02,   5.58438918e-01,   3.21310496e+00,
         2.18812440e+01])

In [833]:
# Normalizo el vector Ci mediante la matriz de overlap

def norm(C,S):
    suma = 0
    for p in range(nsize):
        for q in range(nsize):
            suma += C[p]*S[p,q]*C[q]
    return suma

norma = norm(Ci,Spq)
for i in range(nsize):
    Ci[i] /= np.sqrt(norma)
    
Ci

array([-0.06492225, -0.13068511, -0.0701039 , -0.11847642])

In [834]:
norm(Ci,Spq)

In [835]:
# Calculo de la energia del fundamental

E = 0
for p in range(nsize):
    for q in range(nsize):
        E += (2*Hpq[p,q] + F2[p,q])*Ci[p]*Ci[q]
        
Efund.append(E)

In [836]:
Efund

In [18]:
## Interacion automatica

for i in range(20):
    for p in range(nsize):
        for q in range(p,nsize):
            suma = 0
            for r in range(nsize):
                for s in range(nsize):
                    suma += Qprqs(p,r,q,s,alpha)*Ci[r]*Ci[s]
            F2[p,q] = suma
            F2[q,p] = F2[p,q]

    for p in range(nsize):
        for q in range(p,nsize):
            Fpq[p,q] = Hpq[p,q] + F2[p,q]
            Fpq[q,p] = Fpq[p,q]

    Energias, Cnuevo = eigh(Fpq,Spq,type=1)
    Ci = Cnuevo[:,0]

    E = 0
    for p in range(nsize):
        for q in range(nsize):
            E += (2*Hpq[p,q] + F2[p,q])*Ci[p]*Ci[q]

    Efund.append(E)

In [19]:
Efund

[-0.99855681133497098,
 -0.36971334494202512,
 -0.49082551591837598,
 -0.43923527607800328,
 -0.46137436768614049,
 -0.45184632048889606,
 -0.45594628853069735,
 -0.45418162322500755,
 -0.4549410972221839,
 -0.45461422389582756,
 -0.45475490625213733,
 -0.4546943578571237,
 -0.45472041725798773,
 -0.45470920154875538,
 -0.45471402867694055,
 -0.45471195112916934,
 -0.45471284528489975,
 -0.45471246044922353,
 -0.45471262607864238,
 -0.45471255479339923]